# Import des librairies, dataframes

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import plotly.express as px
import plotly
import plotly.graph_objs as go
from statsmodels.stats.outliers_influence import variance_inflation_factor

import pylab 
import math

import statsmodels.formula.api as smf
import statsmodels.api as sm
import seaborn as sns
import scipy.stats as stats
from statsmodels.stats.stattools import durbin_watson

# pour afficher dynamiquement dans le notebook
from IPython.display import clear_output
from IPython.display import display, Math, Markdown
import time

from datetime import datetime

pd.set_option('display.max_columns', None) 

In [ ]:
echantillon = pd.read_csv("données/echantillon.csv")
echantillon = echantillon.drop(columns=['Unnamed: 0'])
data_ok = pd.read_csv("données/data_ok.csv")
data_ok = data_ok.drop(columns=['Unnamed: 0'])
gini_4 = pd.read_csv("données/gini_4.csv")
dist_revenus_ok = pd.read_csv("données/dist_revenus_ok.csv")
dist_revenus_ok = dist_revenus_ok.drop(columns=['Unnamed: 0'])
dist_revenus_ok = dist_revenus_ok.rename(columns = {"country" : "iso3"})

In [ ]:
dist_rev_moy = dist_revenus_ok.groupby(by = "iso3").mean()
dist_rev_moy = dist_rev_moy.rename(columns = {"income": "income_mean"}).reset_index()
dist_rev_moy = dist_rev_moy[["iso3", "income_mean"]]

In [ ]:
dist_revenus_ok = pd.merge(left = dist_rev_moy, right = dist_revenus_ok, on = "iso3")

In [ ]:
dist_revenus_ok = dist_revenus_ok[["iso3", "income_mean", "quantile", "income"]]

In [ ]:
dist_revenus_ok

In [ ]:
#Merge du dataset
#dist_revenus_ok = dist_revenus_ok.rename(columns = {"country" : "iso3"})
données_completes = pd.merge(left = data_ok, right = dist_revenus_ok, on = "iso3", how = "outer")
données_completes = données_completes[données_completes['country'].notna()]
WID = données_completes[["country", "quantile", "income", "income_mean", "moy_gini", "elasticite_ok"]]
WID

In [ ]:
WID.shape

In [ ]:
len(WID['country'].unique())

In [ ]:
print(WID.isnull().any())

In [ ]:
print(WID.duplicated().any())
print(WID.duplicated().sum())

# Mission 3

## Définition des fonctions utiles

In [ ]:
from collections import Counter

def generate_incomes(n, pj):
    # On génère les revenus des parents (exprimés en logs) selon une loi normale.
    # La moyenne et variance n'ont aucune incidence sur le résultat final (ie. sur le caclul de la classe de revenu)
    ln_y_parent = st.norm(0,1).rvs(size=n)
    # Génération d'une réalisation du terme d'erreur epsilon
    residues = st.norm(0,1).rvs(size=n)
    return np.exp(pj*ln_y_parent + residues), np.exp(ln_y_parent)

def quantiles(l, nb_quantiles):
    size = len(l)
    l_sorted = l.copy()
    l_sorted = l_sorted.sort_values()
    quantiles = np.round(np.arange(1, nb_quantiles+1, nb_quantiles/size) -0.5 +1./size)
    q_dict = {a:int(b) for a,b in zip(l_sorted,quantiles)}
    return pd.Series([q_dict[e] for e in l])

def compute_quantiles(y_child, y_parents, nb_quantiles):
    y_child = pd.Series(y_child)
    y_parents = pd.Series(y_parents)
    c_i_child = quantiles(y_child, nb_quantiles)
    c_i_parent = quantiles(y_parents, nb_quantiles)
    sample = pd.concat([y_child, y_parents, c_i_child, c_i_parent], axis=1)
    sample.columns = ["y_child", "y_parents", "c_i_child","c_i_parent"]
    return sample

def distribution(counts, nb_quantiles):
    distrib = []
    total = counts["counts"].sum()
    if total == 0 :
        return [0] * nb_quantiles
    for q_p in range(1, nb_quantiles+1):
        subset = counts[counts.c_i_parent == q_p]
        if len(subset):
            nb = subset["counts"].values[0]
            distrib += [nb / total]
        else:
            distrib += [0]
    return distrib   

def conditional_distributions(sample, nb_quantiles):
    counts = sample.groupby(["c_i_child","c_i_parent"]).apply(len)
    counts = counts.reset_index()
    counts.columns = ["c_i_child","c_i_parent","counts"]
    mat = []
    for child_quantile in np.arange(nb_quantiles)+1:
        subset = counts[counts.c_i_child == child_quantile]
        mat += [distribution(subset, nb_quantiles)]
    return np.array(mat) 

def plot_conditional_distributions(p, cd, nb_quantiles):
    #plt.figure(figsize=(10,10))
    # La ligne suivante sert à afficher un graphique en "stack bars", sur ce modèle : https://matplotlib.org/gallery/lines_bars_and_markers/bar_stacked.html
    cumul = np.array([0] * nb_quantiles)
    for i, child_quantile in enumerate(cd):
        plt.bar(np.arange(nb_quantiles)+1, child_quantile, bottom=cumul, width=0.95, label = str(i+1) +"e")
        cumul = cumul + np.array(child_quantile)
    plt.axis([.5, nb_quantiles*1.3 ,0 ,1])
    plt.title("p=" + str(p))
    plt.legend(ncol=1)
    plt.xlabel("quantile parents")
    plt.ylabel("probabilité du quantile enfant")
    plt.show()
    
def proba_cond(c_i_parent, c_i_child, mat):
    return mat[c_i_child, c_i_parent]

In [ ]:
def smooth(x,y, box_percent=0.05,res=50,median=True):
    surface = max(x)-min(x)
    my_pas = np.arange(min(x),max(x),surface/res)
    box = surface*box_percent
    demi_box = box/2
    y_sortie = np.array([])
    x_sortie = np.array([])
    for myx in my_pas :
        temp = [y[i] for i in range(len(x)) if ((x[i]>=(myx-demi_box))and(x[i]<=(myx+demi_box)))]
        if median==True :
            temp_y = np.median(temp)
        else :
            temp_y = np.mean(temp)
        #print(temp_y)
        y_sortie = np.append(y_sortie,temp_y)
        #print(y_sortie)
        x_sortie = np.append(x_sortie,myx)
    return x_sortie, y_sortie

### Question 1 à 6

In [ ]:
%%time

#Question 1 à 2 :
pj = 0.9
nb_quantiles = 10
n = 1000*nb_quantiles

#Question 3 : 
y_child, y_parents = generate_incomes(n, pj)

#Question 4 :
sample = compute_quantiles(y_child, y_parents, nb_quantiles)

#Question 5 :
cd = conditional_distributions(sample, nb_quantiles)

#plot_conditional_distributions(pj, cd, nb_quantiles) #prend beaucoup de temps

c_i_child = 5
c_i_parent = 8
p = proba_cond(c_i_parent, c_i_child, cd)
print("\nP(c_i_parent = {} | c_i_child = {}, pj = {}) = {}".format(c_i_parent, 
                                                                   c_i_child,pj, p))

### Question 6

In [ ]:
pj_2 = 0.9 
nb_quantiles_2 = 10
n_2 = 1000*nb_quantiles_2
y_child_2, y_parents_2 = generate_incomes(n_2, pj_2)
sample_2 = compute_quantiles(y_child_2, y_parents_2, nb_quantiles_2)
cd_2 = conditional_distributions(sample_2, nb_quantiles_2)
plot_conditional_distributions(pj_2, cd_2, nb_quantiles_2) # Cette instruction prendra du temps si nb_quantiles > 10

In [ ]:
pj_3 = 0.1 
nb_quantiles_3 = 10
n_3 = 1000*nb_quantiles_3
y_child_3, y_parents_3 = generate_incomes(n_3, pj_3)
sample_3 = compute_quantiles(y_child_3, y_parents_3, nb_quantiles_3)
cd_3 = conditional_distributions(sample_3, nb_quantiles_3)
plot_conditional_distributions(pj_3, cd_3, nb_quantiles_3) # Cette instruction prendra du temps si nb_quantiles > 10

### Question 7

In [ ]:
#On supprime les individus créés, on ne garde que les distributions conditionnelles "cd"
del pj, nb_quantiles, n, y_child, y_parents, sample, c_i_child, c_i_parent, p, cd

### Question 8

In [ ]:
#Multiplication par 500
data_cloned = pd.concat([WID]*500, ignore_index=True)

print('WID shape :', WID.shape)
print('data_cloned shape :', data_cloned.shape)

In [ ]:
for col in data_cloned.columns:
    print(col)

In [ ]:
data_cloned.head(2)

In [ ]:
data_cloned = data_cloned[["country", "quantile", "income", "moy_gini", "elasticite_ok"]]
data_cloned.rename(columns={'quantile': 'c_i_child', 'income': 'y_child', 'moy_gini': 'G_j', 'elasticite_ok': 'p_j'}, inplace=True)

data_cloned.head(2)

In [ ]:
data_cloned.sample(5)

### Question 9

In [ ]:
country_list = data_cloned['country'].unique()
len(country_list)

In [ ]:
# creation d'un variable quantile parents, à remplir
list_proba = []

In [ ]:
%%time

#Pour chaque pays dans la liste
for country in country_list :
    
    #coef d'elasticité pour chaque pays de data_cloned dans la liste country_list, premiere ligne
    pj = data_cloned.loc[data_cloned['country'] == country,'p_j'].iloc[0] 
    
    #nombre de quantiles (nombre de classes de revenu)
    nb_quantiles = 100 
    
    #taille de l'échantillon
    n = 50000 
    
    #Génération de revenus selons une loi normale
    y_child, y_parents = generate_incomes(n, pj) 
    
    #Retourne un df avec y_child, y_parents, c_i_child, c_i_parents
    sample = compute_quantiles(y_child, y_parents, nb_quantiles) 
    
    #Calcul & attribution des probabilités conditonnelles 
    cd = conditional_distributions(sample, nb_quantiles)
    
    #On compte chaque combinaison c_i_child, c_i_parent
    for c_i_child in range(100):
        for c_i_parent in range(100):
            p = proba_cond(c_i_parent, c_i_child, cd)
            #print("\nP(c_i_parent = {} | c_i_child = {}, pj = {}) = {}".format(c_i_parent, c_i_child,pj,p))
            
            #Association des probas conditionelles aux individus
            list_proba.extend([c_i_parent+1]*(int(p*500)))
            

In [ ]:
len(sample)

In [ ]:
sample.head()

In [ ]:
# Mesure de la mobilité
p

In [ ]:
# Je check que ma liste à le meme nombre de ligne que mon data_cloned
len(list_proba)

In [ ]:
# Je créé une colonne dans mon data_cloned pour la classe parents
data_cloned['proba'] = list_proba
data_cloned = data_cloned.rename({'proba':'c_i_parent'},axis=1)
data_cloned.head()

In [ ]:
# Insertion du revenu moyen par pays
mean_income = data_cloned.groupby(by='country').mean()
mean_income.reset_index(inplace=True)
mean_income = mean_income[['country', 'y_child']]
mean_income.rename(columns={'y_child': 'm_j'}, inplace=True)

mean_income.head(2)

 #Insertion du revenu moyen par pays
mean_income = WID[["country", "income_mean"]].drop_duplicates().reset_index()
mean_income.rename(columns={'income_mean': 'm_j'}, inplace=True)
mean_income = mean_income[["country", "m_j"]]
mean_income.head()

In [ ]:
# Jointure de mean_income avec data_cloned & suppression de c_i_child
data_cloned_b = pd.merge(data_cloned, mean_income, on='country')
data_cloned_b = data_cloned_b[["country", "y_child", "G_j", "p_j", "c_i_parent", "m_j"]]

In [ ]:
# Insertion de colonnes log de y_child et m_j
data_cloned_b["income_log"] = np.log(data_cloned_b["y_child"])
data_cloned_b["m_j_log"] = np.log(data_cloned_b["m_j"])

In [ ]:
data_cloned_b.loc[data_cloned_b["country"] == "France"]

In [ ]:
data_cloned_b.info()

In [ ]:
# Sauvegarde du dataframe
#data_cloned_b.to_csv("données/data_cloned_b.csv", index=False)

In [ ]:
data_cloned_b.sample(5)

# Mission 4

In [ ]:
echantillon_m4 = data_cloned_b.loc[(data_cloned_b["country"] == "France")
                     | (data_cloned_b["country"] == "Chile")
                     | (data_cloned_b["country"] == "Iceland") 
                     | (data_cloned_b["country"] == "Paraguay")
                     | (data_cloned_b["country"] == "Vietnam") 
                     | (data_cloned_b["country"] == "Congo, Dem. Rep.")
                    ]

In [ ]:
echantillon_m4

In [ ]:
df = echantillon_m4
fig = px.box(df, x="y_child", y="country", color="country")

fig.show()

print("Les distributions des revenus des individus varient fortement entre les pays dans l'échantillon choisi")

In [ ]:
df = echantillon_m4
fig = px.box(df, x="income_log", y="country", color="country")

fig.show()

print("Les distributions des revenus en log des individus varient fortement entre les pays dans l'échantillon choisi")

In [ ]:
plt.figure(figsize=(30,5))
rs = plt.plot(data_cloned_b.groupby('country').y_child.mean(), 'o')
rs = plt.xticks(rotation=90)
plt.xlabel('pays')
plt.ylabel('revenu moyen des enfants')

In [ ]:
plt.figure(figsize=(30,5))
rs = plt.plot(data_cloned_b.groupby('country').income_log.mean(), 'o')
rs = plt.xticks(rotation=90)
plt.xlabel('pays')
plt.ylabel('revenu moyen des enfants')

In [ ]:
# Agrégation pour gagner en temps de calcul
data_cloned_c = data_cloned_b.groupby(by=['country',
                                  'y_child',
                                  'm_j',
                                  'G_j',
                                  'p_j',
                                  'm_j_log',
                                  'income_log']).mean()
data_cloned_c.reset_index(inplace=True)
#data_projet7_m4_2.drop(columns=['c_i_parent'], inplace=True) # variable non necessaire pour la suite

data_cloned_c.c_i_parent = data_cloned_c.c_i_parent.round()

In [ ]:
print('data_cloned_c shape :', data_cloned_c.shape)
print('data_cloned_b shape :', data_cloned_b.shape)

In [ ]:
data_cloned_c.sample(5)

## Anova

In [ ]:
%%time

anova_pays_rar = smf.ols('y_child~country', data=data_cloned_c).fit()
print(anova_pays_rar.summary())

La pvalue du F test (test de Fisher) étant proche de 0, on peut au niveau de test de 5% rejeter H0 et admettre H1: ainsi on peut conclure que le pays d'origine influe bien sur les revenus. </br>
Par ailleurs, d'après ce modèle, le pays d'origine explique **50%** de la variance du revenu.

### Tests

**Test 1 : Distribution des résidus** </br>
On vérifie l'adéquation de la distribution des résidus de l'ANOVA à une loi normale. l'hypothèse nulle H0 étant que les résidus suivent une loi normale, on procède à un test de Kolmogorov-Smirnov sur les résidus.

In [ ]:
x = anova_pays_rar.resid
stats.kstest(x, 'norm')

HO : les données suivent une loi normale </br>
H1 : les données ne suivent pas une loi normale </br>
pvalue < 0,05 : on rejette H0, les données ne suivent pas une loi normale

Le KS test affichant une pvalue nulle, les données ne suivent pas une loi normale. </br>
On observe la distribution des résidus sur une droite de Henry.

In [ ]:
stats.probplot(x, dist="norm", plot=pylab)
pylab.show()

On a graphiquement une part élevée de résidus s'éloignant de la droite théorique, donc on peut dire que les résidus ne suivent vraisemblablement pas une loi normale.

**Test 2 : Homoscédasticité** </br>
On observe la variance des résidus.
HO l'hypothèse nulle d'hétéroscédasticité des résidus, on réalise un test de Breusch-Pagan sur les résidus.

H0 : homoscédasticité
H1 : hétéroscédasticité

In [ ]:
print(sm.stats.diagnostic.het_breuschpagan(anova_pays_rar.resid, anova_pays_rar.model.exog))

Avec une p value proche de 0 (seconde valeur), on admet l'hétéroscédasticité. </br>
On peut observer les variances des résidus sur le nuage de variance résiduelle

In [ ]:
ax=plt.plot(anova_pays_rar.fittedvalues, anova_pays_rar.resid , ".",  alpha=0.3)
plt.title("Nuage de la variance résiduelle", fontsize=18)
#plt.xlabel("GWh", fontsize=16), plt.ylabel("Résidus", fontsize=16)

On peut confirmer la non linéarité avec un **test de Rainbow**, qui vérifie H0 : la représentation statistique est bien linéaire. </br> 
La p-value renvoyée par ce test devrait donc être **supérieure à 0,05** pour qu'on considère que le modèle de régression peut être conservé. </br>
Ici, la p value étant inférieure au seuil, on rejette H0. 

In [ ]:
from statsmodels.stats.diagnostic import linear_rainbow
Ftest, pval = linear_rainbow(anova_pays_rar)
print(pval)

En conclusion, si ce modèle explique bien 50% de la variance du revenu des individus avec le pays d'origine, il ne semble pas être robuste aux tests usuels et ne peut donc être utilisé pour une prédiction pertinente.

## Régression linéaire avec le revenu moyen et l'indice de gini

### Modèle 1 : revenu & gini

In [ ]:
# Variables explicatives : revenu moyen du pays de l’individu et l’indice de Gini du pays de l’individu
reg_v1_1 = smf.ols('y_child ~ G_j+m_j', data=data_cloned_c).fit()
print(reg_v1_1.summary())

In [ ]:
reg_v1_1_sct = reg_v1_1.centered_tss
reg_v1_1_sce = reg_v1_1.ess
reg_v1_1_scr = reg_v1_1.ssr
reg_v1_1_r2 = reg_v1_1.rsquared

display(Markdown(f"""
Décomposition de la variance : <br>

**SCT = SCE + SCR**
- SCT = {reg_v1_1_sct:.0f} <br>
- SCE = {reg_v1_1_sce:.0f} <br>
- SCR = {reg_v1_1_scr:.0f} <br>

$R^2 = {reg_v1_1_r2:.2f}$

Le modèle explique **{reg_v1_1_r2*100:.0f}% de la variance totale**.
"""))

Selon ce modèle, le pays de naissance (ie. le revenu moyen et l’indice de Gini) explique donc 50% de la variance totale, tandis que les autres facteurs non considérés dans le modèle (efforts, chance, etc.) représentent l'autre moitié.
Par ailleurs, l'indice de Gini n'est pas statistiquement significatif au seuil de 0,05.

### Modèle 2 : revenus en log & indice de gini

In [ ]:
# Variables explicatives : revenu moyen du pays de l’individu en log et l’indice de Gini du pays de l’individu
reg_v1_2 = smf.ols('income_log ~ G_j+m_j_log', data=data_cloned_c).fit()

In [ ]:
reg_v1_2_sct = reg_v1_2.centered_tss
reg_v1_2_sce = reg_v1_2.ess
reg_v1_2_scr = reg_v1_2.ssr
reg_v1_2_r2 = reg_v1_2.rsquared

display(Markdown(f"""
Décomposition de la variance : <br>

**SCT = SCE + SCR**
- SCT = {reg_v1_2_sct:.0f} <br>
- SCE = {reg_v1_2_sce:.0f} <br>
- SCR = {reg_v1_2_scr:.0f} <br>

$R^2 = {reg_v1_2_r2:.2f}$

Le modèle explique **{reg_v1_2_r2*100:.0f}% de la variance totale**.
"""))

Selon ce modèle, le pays de naissance (ie. le revenu moyen et l’indice de Gini) explique donc **73%** de la variance totale, tandis que les autres facteurs non considérés dans le modèle (efforts, chance, etc.) représentent les **27%** restant.
Le passage en log des variables de revenus améliore le pouvoir explicatif du modèle, et l'indice de Gini devient significatif. Cependant, l'interprétation est rendue plus compliquée du fait des logs (pas de "lecture naturelle" posible).

## Régression linéaire avec le revenu moyen, l'indice de gini et la classe de revenu des parents

### Modèle 3 : revenu, gini & classe de revenus

In [ ]:
# Variables explicatives : revenu moyen du pays de l’individu indice de Gini du pays de l’individu, classe de revenu des parents
reg_v2_1 = smf.ols('y_child ~ G_j+m_j+c_i_parent', data=data_cloned_c).fit()
print(reg_v2_1.summary())

In [ ]:
reg_v2_1_sct = reg_v2_1.centered_tss
reg_v2_1_sce = reg_v2_1.ess
reg_v2_1_scr = reg_v2_1.ssr
reg_v2_1_r2 = reg_v2_1.rsquared

display(Markdown(f"""
Décomposition de la variance : <br>

**SCT = SCE + SCR**
- SCT = {reg_v2_1_sct:.0f} <br>
- SCE = {reg_v2_1_sce:.0f} <br>
- SCR = {reg_v2_1_scr:.0f} <br>

$R^2 = {reg_v2_1_r2:.2f}$

Le modèle explique **{reg_v2_1_r2*100:.0f}% de la variance totale**.
"""))

Selon ce modèle, le pays de naissance (ie. le revenu moyen, la classe de revenus des aprents, et l’indice de Gini) explique donc **65%** de la variance totale, tandis que les autres facteurs non considérés dans le modèle (efforts, chance, etc.) représentent les **35%** restant. </br>
Cependant, et comme pour le premier modèle, l'indice de gini n'est pas statistiquement significatif au seuil de 0,05.

### Modèle 4 : revenus en log, gini & classes de revenus

In [ ]:
# Variables explicatives : revenu moyen du pays de l’individu en log, 
 # l'indice de Gini du pays de l’individu, classe de revenu des parents
reg_v2_2 = smf.ols('income_log ~ G_j+m_j_log+c_i_parent', data=data_cloned_c).fit()

In [ ]:
reg_v2_2_sct = reg_v2_2.centered_tss
reg_v2_2_sce = reg_v2_2.ess
reg_v2_2_scr = reg_v2_2.ssr
reg_v2_2_r2 = reg_v2_2.rsquared

display(Markdown(f"""
Décomposition de la variance : <br>

**SCT = SCE + SCR**
- SCT = {reg_v2_2_sct:.0f} <br>
- SCE = {reg_v2_2_sce:.0f} <br>
- SCR = {reg_v2_2_scr:.0f} <br>

$R^2 = {reg_v2_2_r2:.4f}$

Le modèle explique **{reg_v2_2_r2*100:.0f}% de la variance totale**.
"""))

Le modèle le plus performant est celui expliquant le **revenu des enfants en log** par le **revenu du pays en log**, **l'indice de gini** et la **classe de revenus des parents**, avec un **$R^2 = 0,96$**.<br>
On risque cependant de faire face à un problème de **sur-ajustement du modèle**, et une complication de l'interprétation du fait de l'utilisation des log.

In [ ]:
print("Le R^2 ajusté est similaire, avec une valeur de ", round(reg_v2_2.rsquared_adj, 4))

## Tests

In [ ]:
#Récapitulatif :
R2 = [reg_v1_1_r2, reg_v1_2_r2,
     reg_v2_1_r2, reg_v2_2_r2]#,reg_v3_1_r2, reg_v3_2_r2]
reg_name = ["Modèle 1", "Modèle 1_2",
           "Modèle 2", "Modèle 2_2"]#,"Modèle 3", "Modèle 3_2"]
log = ["Non", "Oui", "Non", "Oui"]#, "Oui", "Oui"]
var = ['y_child ~ G_j+m_j',
       'income_log ~ G_j+m_j_log',
       'y_child ~ G_j+m_j+c_i_parent',
       'income_log ~ G_j+m_j_log+c_i_parent']#,'income_log ~ m_j_log + G_j','income_log ~ m_j_log + G_j+c_i_parent']
list_of_tuples = list(zip(reg_name, var, R2, log))
list_of_tuples 

df = pd.DataFrame(list_of_tuples,
                  columns = ['Modèle', 'Variables', 'R2', 'Logarithme'])
df.R2 = df.R2.round(2)
df

On va conserver le modèle 2_2, utilisant à la fois les revenus, l'indice de Gini et la classe de revenu des parents, avec des variables non log, qui permettent une interprétation facilitée.
On pourra également aborder le modèle.

### Calcul des leviers

In [ ]:
# Paramètres de l'étude
n = data_cloned_c.shape[0] # échantillon
p = 4 # nombre de variables

# Seuil levier selon Belsey
seuil_levier = 2 * p/n
seuil_levier

In [ ]:
# Ajout des leviers
data_cloned_c['Leviers'] = reg_v2_2.get_influence().hat_matrix_diag

Leviers_sup = data_cloned_c.loc[data_cloned_c['Leviers'] > seuil_levier, :]
Leviers_inf = data_cloned_c.loc[data_cloned_c['Leviers'] <= seuil_levier, :]

Leviers_sup.head().style.format({"Leviers": "{:,.7f}"}) # permet d'afficher pour la colonne choisie le nombre de chiffre apres la virgules

In [ ]:
len(Leviers_sup['country'].unique())

On a donc 9 pays présentant des observations au dessus du seuil

In [ ]:
%%time

# Représentation des leviers
plt.figure(figsize=(10,6))

# Individus sous le seuil
plt.bar(Leviers_inf['country'],Leviers_inf['Leviers'])

# Individus au dessus du seuil
plt.bar(Leviers_sup['country'],Leviers_sup['Leviers'])

# Décoration et annotations
plt.title('Représentation des leviers', fontsize=22)
plt.xlabel('Pays', fontsize=18)
plt.xticks('', fontsize=16)
plt.ylabel('Leviers', fontsize=18)
plt.yticks(fontsize=16)
plt.axhline(y=seuil_levier, linestyle='-')
plt.text(50, 0.00085 , 'Seuil levier', fontsize = '14', color='red')
plt.tight_layout()
plt.show()

In [ ]:
country_lev = Leviers_sup['country'].unique()
country_lev

### Résidus studentisés : outliers

Le seuil pour les résidus studentisés est une **loi de Student** à **n-p-1 degrés de liberté**

In [ ]:
from scipy.stats import t, shapiro
alpha = 0.05
data_cloned_c['rstudent'] = reg_v2_2.get_influence().resid_studentized_internal
seuil_rstudent = t.ppf(1-alpha/2,n-p-1)
seuil_rstudent

In [ ]:
# statistique de test par observation
data_cloned_c.sort_values(by='rstudent').head()

In [ ]:
data_cloned_c.sort_values(by='rstudent').tail()

In [ ]:
rstudent_in = data_cloned_c.loc[(data_cloned_c['rstudent'] <= seuil_rstudent) & (data_cloned_c['rstudent'] >= -seuil_rstudent), :]
rstudent_sup = data_cloned_c.loc[(data_cloned_c['rstudent'] > seuil_rstudent), :]
rstudent_inf = data_cloned_c.loc[(data_cloned_c['rstudent'] < -seuil_rstudent), :]

In [ ]:
%%time

# Représentation des résidus studentisés
plt.figure(figsize=(10,10))

# Individus entre le seuil mini et le seuil maxi
plt.bar(rstudent_in['country'],rstudent_in['rstudent'], color='orange')

# Individus au dessus du seuil maxi
plt.bar(rstudent_sup['country'],rstudent_sup['rstudent'], color='steelblue')

# Individus en dessous du seuil maxi
plt.bar(rstudent_inf['country'],rstudent_inf['rstudent'], color='steelblue')

# Annotations
plt.title('Représentation des résidus studentisés', fontsize=22)
plt.xlabel('Pays', fontsize=18)
plt.xticks('', fontsize=16)
plt.ylim(-22,9)
plt.ylabel('Résidus studentisés', fontsize=18)
plt.yticks(fontsize=16)

plt.axhline(y=seuil_rstudent, color='steelblue', linestyle='-')
plt.text(50, 6.2 , 'Seuil rstudent', fontsize = '18', color='steelblue')

plt.axhline(y=-seuil_rstudent, color='steelblue', linestyle='-')
plt.text(50, -6.2 , '-Seuil rstudent', fontsize = '18', color='steelblue')

plt.tight_layout()
plt.show()

In [ ]:
# Nombre de valeurs atypiques sur les variables à expliquer
res_stu_ln = data_cloned_c.loc[(data_cloned_c['rstudent'] > seuil_rstudent) | (data_cloned_c['rstudent'] < -seuil_rstudent)]

len(res_stu_ln)

In [ ]:
# Pays presentants des valeurs atypiques
outliers_country = _country = data_cloned_c.loc[data_cloned_c.index.isin(res_stu_ln.index)].groupby(by='country').count().sort_values(by='y_child',ascending=False)
outliers_country 

### Distance de cook

In [ ]:
# Création du dataframe avec tous les resultats d'influences disponibles
influence_ln = reg_v2_2.get_influence().summary_frame()
influence_ln.head()

In [ ]:
# Ajout de la colonne distance de Cook à notre dataframe analyses_ln
data_cloned_c['dcooks'] = influence_ln['cooks_d']
# Seuil d'influence selon Cook
seuil_dcook = 4/(n-p)
data_cloned_c.sort_values(by='dcooks').head().style.format({"dcooks": "{:,.13f}"})

In [ ]:
data_cloned_c.sort_values(by='dcooks').tail().style.format({"dcooks": "{:,.7f}"})

In [ ]:
cooks_sup = data_cloned_c.loc[(data_cloned_c['dcooks'] > seuil_dcook), :]
cooks_inf = data_cloned_c.loc[(data_cloned_c['dcooks'] <= seuil_dcook), :]

In [ ]:
%%time

# Représentation de la distances de Cooks
plt.figure(figsize=(10,10))

# Individus sous le seuil
plt.bar(cooks_inf['country'],cooks_inf['dcooks'], color='blue')

# Individus au dessus le seuil
plt.bar(cooks_sup['country'],cooks_sup['dcooks'], color='orange')

# Décoration et annotations
plt.title('Représentation de la distance de Cook', fontsize=22)
plt.ylabel('Distance de Cook', fontsize=18)
plt.xticks('', fontsize=16)
plt.xticks(fontsize=16)

plt.text(-5, (seuil_dcook + 0.00300), 'Seuil de Cook', fontsize = '18', color='steelblue')
plt.axhline(y=seuil_dcook, color='steelblue', linestyle='-')

plt.tight_layout()
plt.show()

In [ ]:
# Nombre d'observations influentes
dco_ln = data_cloned_c.loc[data_cloned_c['dcooks'] > seuil_dcook]
len(dco_ln)

In [ ]:
# Pays présentants des obervations au dela du seuil
data_cloned_c.loc[data_cloned_c.index.isin(dco_ln.index)].groupby(by='country').count().sort_values(by='y_child',ascending=False)

In [ ]:
# Obeservations atypiques et influentes
ind_aty_infl_ln = data_cloned_c.loc[((data_cloned_c['dcooks'] > seuil_dcook) &
                                  (data_cloned_c['rstudent'] > seuil_rstudent) &
                                  (data_cloned_c['Leviers'] > seuil_levier)) |
                                  ((data_cloned_c['dcooks'] > seuil_dcook) &
                                  (data_cloned_c['rstudent'] < -seuil_rstudent) &
                                  (data_cloned_c['Leviers'] > seuil_levier))]
len(ind_aty_infl_ln)

In [ ]:
susp_pt = data_cloned_c.loc[data_cloned_c.index.isin(ind_aty_infl_ln.index)]
susp_pt.reset_index(inplace=True)
susp_pt.groupby('country').count()

In [ ]:
susp_pt.groupby('country').count().sort_values(by = "index",ascending=False)

### Valeurs influentes

from statsmodels.graphics.regressionplots import *
influence_plot(reg_v2_2) # myreg doit être un model de statsmodels

#plt.xlim(0,0.002) # paramétrage manuel

plt.show()

### Colinéarité des variables

In [ ]:
variables = reg_v2_2.model.exog
[variance_inflation_factor(variables, i) for i in np.arange(1,variables.shape[1])]

### Homoscédasticité des résidus

In [ ]:
_, pval, __, f_pval = sm.stats.diagnostic.het_breuschpagan(reg_v2_2.resid, variables)
print('p value test Breusch Pagan:', pval)

### Normalité des résidus

In [ ]:
shapiro(reg_v2_2.resid)

In [ ]:
stats.kstest(reg_v2_2.resid, 'norm')

In [ ]:
stats.probplot(reg_v2_2.resid, dist="norm", plot=pylab)
pylab.show()

In [ ]:
plt.hist(reg_v2_2.resid)

In [ ]:
#Distribution des résidus
fig, ax = plt.subplots(1, 2, figsize=(20,10))

plt.hist(reg_v2_2.resid, density=True)

model_norm_residuals = reg_v2_2.get_influence().resid_studentized_internal
QQ = sm.ProbPlot(model_norm_residuals)
QQ.qqplot(line='45', alpha=0.5, color='#4C72B0', ax=ax[0])

ax[0].set_title('Q-Q Plot')
ax[1].set_title('Histogramme des résidus')
ax[1].set_xlabel('Valeurs résiduelles')
ax[1].set_ylabel('Nombre de résidus')
                
plt.show()

In [ ]:
#Homoscédasticité

print(sm.stats.diagnostic.het_breuschpagan(reg_v2_2.resid, reg_v2_2.model.exog))

ax=plt.plot(reg_v2_2.fittedvalues, reg_v2_2.resid, ".",  alpha=0.3)
plt.title("Nuage de la variance résiduelle", fontsize=18)
#plt.xlabel("GWh", fontsize=16), plt.ylabel("Résidus", fontsize=16)

from statsmodels.stats.diagnostic import linear_rainbow
Ftest, pval = linear_rainbow(reg_v2_2)
print(pval)

### Prédiction

In [ ]:
country = data_cloned_c['country'].unique()
country

In [ ]:
pays_selectionné = 'France'
c_i_parent_target = 50.0

print('Le pays selectionné est :', pays_selectionné)

revenu_moyen_pays = data_cloned_c.loc[(data_cloned_c['country'] == pays_selectionné), 'm_j'].iloc[0]
print('Le revenu moyen du pays selectionné est de', round(revenu_moyen_pays, 2),"$")

revenu_moyen_pays_log = np.log(revenu_moyen_pays)
print('Le revenu moyen en log du pays selectionné est de', round(revenu_moyen_pays_log, 3))

indice_gini_pays = data_cloned_c.loc[(data_cloned_c['country'] == pays_selectionné), 'G_j'].iloc[0]
print("L'indice de gini du pays selectionné est de", round(indice_gini_pays))

a_prevoir = pd.DataFrame({'m_j_log':[revenu_moyen_pays_log], 'G_j':[indice_gini_pays],'c_i_parent':[c_i_parent_target]})
Revenu_enfant = reg_v2_2.predict(a_prevoir)

# Ajouter * devant une variable lors d'un print permet de ne pas afficher le Dtypes ni l'index
Revenu_enfant_calculé = np.exp(*Revenu_enfant)
Revenu_enfant_calculé

print("Le revenu d'un individu en",pays_selectionné,
      "dont les parents ont une classe de revenu égale à",c_i_parent_target,
      "est de",round(Revenu_enfant_calculé, 2),"$")

In [ ]:
f = data_cloned_c.loc[(data_cloned_c['country'] == pays_selectionné)]
g = f.loc[f["c_i_parent"] == c_i_parent_target]
h = g.y_child.mean()
print(round(h, 2))

In [ ]:
%whos DataFrame

In [ ]:
del data_cloned
del data_cloned_b
del data_cloned_c
del WID
del Leviers_inf
del dist_revenus_ok
del echantillon_m4
del influence_ln
del rstudent_in
del sample
del sample_2
del sample_3

# FIN ICI